# Define Classes and create the SQLite database

> This module defines the classes we use to represent the PKM workflow.

In [ ]:
#| default_exp classdb

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from __future__ import annotations
import json
from enum import Enum
from typing import Union, ClassVar
from dataclasses import dataclass
from pydantic import BaseModel, field_serializer, field_validator, Field, computed_field
from fastlite import *
from fastcore.test import *
from hopsa import ossys

We use `from __future__ import annotations` to support forward references in type hints. To be precise in the `@classmethod` we create to keep track of all instances of the class.

## Using Pydantic with MiniDataAPI and SQLite

We want to use `Pydantic` Dataclasses to enable typechecking and validation. We also want to use the Dataclasses with the `MiniDataAPI` to create the tables in the `SQLite` database. But `SQLite` only has datatypes: `NULL`, `INTEGER`, `REAL`, `TEXT`, and `BLOB`. So no `list` or any of the Dataclass(Enum) types we use.

To be able to use both `Pydanctic` and the `MiniDataAPI` we will do two things:

1. Define a Pydantic Dataclass with the correct datatypes and a Dataclass that has the same fields as the Pydantic Dataclass, but with datatypes that can be used with SQLite.
2. We add `@field_serializer` methods to the Pydantic Dataclass that convert the fields to JSON strings when we use the method `.model_dump()` on the instance of the Pydantic Dataclass. These serialised JSON strings can then be added to the SQLite database.
3. We also use the `@field_validator` decorator to convert the JSON strings back to the correct datatypes when we load the data from the SQLite database back into the Pydantic Dataclass.

This way we can:

- create instances with the Pydantic Dataclass to have easy typechecking and validation.
- convert this instances to `MiniDataAPI` and `SQLite` friendly datatypes using `.model_dump()` on the instance, that we can then add to the database.
- load the data from the SQLite database back into the Pydantic Dataclass.

The exact implementation can be found below where the Classes are defined.

#### Track instances of classes

We also want to keep track of the instances available for each class. Therefore we need some higher order magic.

- a list in the class to store the instances
- a __init__ method to add the instance to the list
- a classmethod to get the list of instances

We can't just add a `_instances = []` statement to the Class, because Pydantic will then assume it is a model field (private attribute). We need to tell Pydantic to ignore the _instances class variable as a model field and treat is as a class variable. Therefore we need to import `ClassVar` from `typing` and use it to type the _instances variable.

## Enum Classes

First we define the possible values of the different variables that are available in the classes. We use the module `enum` to define **Enumerations**. We use this to bind the possible values to a variable name, making the code more readable and maintainable.

In [ ]:
#| export
class InformationType(Enum):
    """Information content types that flow through the PKM workflow."""
    BOOK = "book"
    RESEARCH_PAPER = "research_paper"
    DOCUMENT = "document"
    ANNOTATION = "annotations&highlights"
    NOTE = "note"
    EMAIL = "email"
    DISCORD_MESSAGE = "discord_message"
    WEB_ARTICLE = "web_article"
    YOUTUBE_VIDEO = "youtube_video"
    PODCAST = "podcast"
    PRODUCT_IDEA = "product_idea"
    PROJECT_IDEA = "project_idea"

class Method(Enum):
    """How actions are performed - manually or automatically."""
    MANUAL = "manual"
    AUTOMATIC = "automatic"

class Phase(Enum):
    """The five phases of the PKM workflow."""
    COLLECT = "collect"
    RETRIEVE = "retrieve"
    CONSUME = "consume"
    EXTRACT = "extract"
    REFINE = "refine"

class PhaseQuality(Enum):
    """Quality rating for how well a tool performs in each phase."""
    NA = "na"
    BAD = "bad"
    OK = "ok"
    GREAT = "great"

class OrganizationSystem(Enum):
    """How tools organize and structure information."""
    TAGS = "tags"
    FOLDERS = "folders"
    LINKS = "links"
    JOHNNY_DECIMAL = "johnny_decimal"

In [ ]:
Phase("refine")

<Phase.REFINE: 'refine'>

## PKM Workflow Classes

Next we create a dataclass for each item we need to be present in the PKM workflow.

**Pydantic Dataclasses**

Used for typechecking.

When creating a new instance for an `InformationItem` the `toolflow` must be given as a list of `Tool` objects. The typechecking makes sure that any `Tool` object mentioned in the `toolfow` list, does exist as an actual `Tool` instance. So make sure to first create  all the `Tool` instances that are needed for an `InformationItem`, before creating the `InformationItem` instance.

:::{.callout-tip}
I had some serious trouble getting the Pydantic dataclass validations to work. One of the issues is described above and is about SQLite not supporting all datatypes. A second major issue is that the Pydantic Dataclasses reference each other. The `InformationItem` references the `Tool` in the `toolflow` field. I would also be convenient to store all the `InformationItem`s that can be used with a certain `Tool`, but in that case we would create a circular reference between `InformationItem` and `Tool`.

We decided to remove the `information_items` list from `Tool`. When we need to get all the `InformationItem`s that are supported by a `Tool` we can write a Python function or do a SQL-query on the SQLite database.

But then we are left with the fact that we want a list of `Tool`s that exist. These are the options considered:

- `toolflow: list[Tool]`
- `toolflow: list[Tool.name]`
- `toolflow: list[str]`

The last option is used in combination with validation to ensure each string is a valid Tool.name.

Here's why this is the best approach:

- Clean serialization (no complex object embedding)
- Human-readable in the database
- Type safety through validation
- Easy to query

The same goes for the `Improvement` class and the field `tool`.
:::

In [ ]:
#| export
class PhaseQualityData(BaseModel):
    collect: PhaseQuality = Field(PhaseQuality.NA)
    retrieve: PhaseQuality = Field(PhaseQuality.NA)
    consume: PhaseQuality = Field(PhaseQuality.NA)
    extract: PhaseQuality = Field(PhaseQuality.NA)
    refine: PhaseQuality = Field(PhaseQuality.NA)

class Tool(BaseModel):
    name: str = Field(..., description="Name of the tool")
    organization_system: list[OrganizationSystem] = Field(..., description="Organization systems supported by the tool")
    phase_quality: PhaseQualityData = Field(..., description="Quality of the tool for each phase")
    collect: str | None = Field(default=None, description="Description how to use tool in collect phase")
    retrieve: str | None = Field(default=None, description="Description how to use tool in retrieve phase")
    consume: str | None = Field(default=None, description="Description how to use tool in consume phase")
    extract: str | None = Field(default=None, description="Description how to use tool in extract phase")
    refine: str | None = Field(default=None, description="Description how to use tool in refine phase")

    @computed_field
    @property
    def slug(self) -> str:
        return ossys.sanitize_name(self.name)

    def flatten_for_db(self):
        base = self.model_dump(exclude={'phase_quality', 'organization_system'})
        base.update({'organization_system': json.dumps([org.value for org in self.organization_system]), 'collect_quality': self.phase_quality.collect.value, 'retrieve_quality': self.phase_quality.retrieve.value, 'consume_quality': self.phase_quality.consume.value, 'extract_quality': self.phase_quality.extract.value, 'refine_quality': self.phase_quality.refine.value})
        return base

    _instances: ClassVar[Dict[str, Tool]] = {}

    def __init__(self, **data):
        super().__init__(**data)
        type(self)._instances[self.slug] = self
    
    @classmethod
    def get_instances(cls) -> Dict[str, Tool]:
        return cls._instances
    
  
    @classmethod
    def from_db(cls, db_record):
        phase_quality = PhaseQualityData(collect=PhaseQuality(db_record['collect_quality']), retrieve=PhaseQuality(db_record['retrieve_quality']), consume=PhaseQuality(db_record['consume_quality']), extract=PhaseQuality(db_record['extract_quality']), refine=PhaseQuality(db_record['refine_quality']))
        org_systems = [OrganizationSystem(s) for s in json.loads(db_record['organization_system'])]
        return cls(name=db_record['name'], organization_system=org_systems, phase_quality=phase_quality, collect=db_record['collect'], retrieve=db_record['retrieve'], consume=db_record['consume'], extract=db_record['extract'], refine=db_record['refine'])

In [ ]:
#| export
class PhaseMethodData(BaseModel):
    collect: Method | None = Field(default=None)
    retrieve: Method | None = Field(default=None)
    consume: Method | None = Field(default=None)
    extract: Method | None = Field(default=None)
    refine: Method | None = Field(default=None)

class PhaseToolflowData(BaseModel):
    collect: Union[str, tuple[str, ...], None] = Field(default=None)
    retrieve: Union[str, tuple[str, ...], None] = Field(default=None)
    consume: Union[str, tuple[str, ...], None] = Field(default=None)
    extract: Union[str, tuple[str, ...], None] = Field(default=None)
    refine: Union[str, tuple[str, ...], None] = Field(default=None)

    @staticmethod
    def _san(v):
        if v is None: return None
        if isinstance(v, str): return ossys.sanitize_name(v)
        if isinstance(v, tuple):
            return tuple([ossys.sanitize_name(i) for i in v])
    
    @field_validator('collect', 'retrieve', 'consume', 'extract', 'refine', mode='before')
    def _val(cls, v): return cls._san(v)


In [ ]:
PhaseToolflowData(
    collect=("Reader", "Recall"),
    retrieve="Recall",
    consume=None,
    extract=None,
    refine=None
)

PhaseToolflowData(collect=('reader', 'recall'), retrieve='recall', consume=None, extract=None, refine=None)

In [ ]:
#| export
class InformationItem(BaseModel):
    name: str = Field(..., description="Name of the information item")
    info_type: InformationType = Field(..., description="Type of information item, e.g. book, article, video, etc.")
    method: PhaseMethodData = Field(..., description="Methods used at each phase")
    toolflow: PhaseToolflowData = Field(..., description="Tools used for this item at each phase")

    @computed_field
    @property
    def slug(self) -> str:
        return ossys.sanitize_name(self.name)

    def flatten_for_db(self):
        base = self.model_dump(exclude={'method', 'toolflow'})
        base.update({'collect_method': self.method.collect.value if self.method.collect else None, 'retrieve_method': self.method.retrieve.value if self.method.retrieve else None, 'consume_method': self.method.consume.value if self.method.consume else None, 'extract_method': self.method.extract.value if self.method.extract else None, 'refine_method': self.method.refine.value if self.method.refine else None, 'collect_toolflow': json.dumps(self.toolflow.collect) if isinstance(self.toolflow.collect, (list, tuple)) else self.toolflow.collect, 'retrieve_toolflow': json.dumps(self.toolflow.retrieve) if isinstance(self.toolflow.retrieve, (list, tuple)) else self.toolflow.retrieve, 'consume_toolflow': json.dumps(self.toolflow.consume) if isinstance(self.toolflow.consume, (list, tuple)) else self.toolflow.consume, 'extract_toolflow': json.dumps(self.toolflow.extract) if isinstance(self.toolflow.extract, (list, tuple)) else self.toolflow.extract, 'refine_toolflow': json.dumps(self.toolflow.refine) if isinstance(self.toolflow.refine, (list, tuple)) else self.toolflow.refine})
        return base

    _instances: ClassVar[Dict[str, InformationItem]] = {}

    def __init__(self, **data):
        super().__init__(**data)
        type(self)._instances[self.slug] = self
    
    @classmethod
    def get_instances(cls) -> Dict[str, InformationItem]:
        return cls._instances
    
    @field_serializer('info_type')
    def db_serialize(self, v):
        return v.value


In [ ]:
#| export
class Improvement(BaseModel):
    title: str = Field(..., description="Title of the improvement")
    what: str = Field(..., description="What needs to be improved")
    why: str = Field(..., description="Why is this improvement needed")
    prio: int = Field(..., description="Priority of the improvement")
    tool: str = Field(..., description="Tool that needs improvement")
    phase: Phase = Field(..., description="Phase that needs improvement")

    @computed_field
    @property
    def slug(self) -> str:
        return ossys.sanitize_name(self.title)

    def flatten_for_db(self):
        return self.model_dump()

    _instances: ClassVar[Dict[str, Improvement]] = {}

    def __init__(self, **data):
        super().__init__(**data)
        type(self)._instances[self.slug] = self
    
    @classmethod
    def get_instances(cls) -> Dict[str, Improvement]:
        return cls._instances
    
    @field_serializer('phase')
    def db_serialize(self, v):
        return v.value
    
    @field_validator('tool')
    def validate_tool_names(cls, v):
        valid_tools = Tool.get_instances().keys()
        if v not in valid_tools: raise ValueError(f"Tool '{v}' does not exist")
        return v

Test creating instances

In [ ]:
def test_phase_quality_data():
    pqd = PhaseQualityData(collect=PhaseQuality.GREAT, retrieve=PhaseQuality.BAD, consume=PhaseQuality.OK, extract=PhaseQuality.NA, refine=PhaseQuality.GREAT)
    test_eq(pqd.collect, PhaseQuality.GREAT)
    test_eq(pqd.retrieve, PhaseQuality.BAD)

def test_tool_creation():
    tool = Tool(name="TestTool", organization_system=[OrganizationSystem.TAGS], phase_quality=PhaseQualityData(collect=PhaseQuality.GREAT, retrieve=PhaseQuality.BAD, consume=PhaseQuality.OK, extract=PhaseQuality.NA, refine=PhaseQuality.GREAT))
    test_eq(tool.name, "TestTool")
    test_eq(tool.slug, "testtool")
    test_eq(tool.phase_quality.collect, PhaseQuality.GREAT)
    return tool

def test_tool_flatten():
    tool = Tool(name="TestTool", organization_system=[OrganizationSystem.TAGS], phase_quality=PhaseQualityData(collect=PhaseQuality.GREAT, retrieve=PhaseQuality.BAD, consume=PhaseQuality.OK, extract=PhaseQuality.NA, refine=PhaseQuality.GREAT))
    flat = tool.flatten_for_db()
    test_eq(flat['collect_quality'], 'great')
    test_eq(flat['retrieve_quality'], 'bad')
    test_eq(flat['name'], 'TestTool')

def test_information_item():
    methods = PhaseMethodData(collect=Method.MANUAL, retrieve=None, consume=None, extract=None, refine=None)
    tools = PhaseToolflowData(collect="Reader", retrieve="Recall", consume=None, extract=None, refine=None)
    item = InformationItem(name="Test Article", info_type=InformationType.WEB_ARTICLE, method=methods, toolflow=tools)
    test_eq(item.method.collect, Method.MANUAL)
    test_eq(item.toolflow.collect, "Reader")
    return item

def test_information_item_flatten():
    methods = PhaseMethodData(collect=Method.MANUAL, retrieve=None, consume=None, extract=None, refine=None)
    tools = PhaseToolflowData(collect=["Reader", "Recall"], retrieve="Recall", consume=None, extract=None, refine=None)
    item = InformationItem(name="Test Article", info_type=InformationType.WEB_ARTICLE, method=methods, toolflow=tools)
    flat = item.flatten_for_db()
    test_eq(flat['collect_method'], 'manual')
    test_eq(flat['retrieve_method'], None)
    test_eq(flat['collect_toolflow'], '["Reader", "Recall"]')
    test_eq(flat['retrieve_toolflow'], 'Recall')

def test_improvement():
    imp = Improvement(title="Fix Search", what="Better search in Reader", why="Current search is bad", prio=1, tool="testtool", phase=Phase.RETRIEVE)
    test_eq(imp.title, "Fix Search")
    test_eq(imp.phase, Phase.RETRIEVE)
    test_eq(imp.flatten_for_db()['phase'], 'retrieve')
    return imp

In [ ]:
test_phase_quality_data()
test_tool_creation()
test_tool_flatten()
test_information_item()
test_information_item_flatten()
test_improvement()

AssertionError: ==:
reader
Reader

In [ ]:
Tool.get_instances()

{'testtool': Tool(name='TestTool', organization_system=[<OrganizationSystem.TAGS: 'tags'>], phase_quality=PhaseQualityData(collect=<PhaseQuality.GREAT: 'great'>, retrieve=<PhaseQuality.BAD: 'bad'>, consume=<PhaseQuality.OK: 'ok'>, extract=<PhaseQuality.NA: 'na'>, refine=<PhaseQuality.GREAT: 'great'>), collect=None, retrieve=None, consume=None, extract=None, refine=None, slug='testtool')}

In [ ]:
InformationItem.get_instances()

{'test_article': InformationItem(name='Test Article', info_type=<InformationType.WEB_ARTICLE: 'web_article'>, method=PhaseMethodData(collect=<Method.MANUAL: 'manual'>, retrieve=None, consume=None, extract=None, refine=None), toolflow=PhaseToolflowData(collect=['Reader', 'Recall'], retrieve='Recall', consume=None, extract=None, refine=None), slug='test_article')}

In [ ]:
Improvement.get_instances()

{'fix_search': Improvement(title='Fix Search', what='Better search in Reader', why='Current search is bad', prio=1, tool='testtool', phase=<Phase.RETRIEVE: 'retrieve'>, slug='fix_search')}

## SQLite database

#### Create and connect

Connect to the database in the `main.py`. We should also enable foreign key constraints. These are disabled by default in Sqlite.

For testing purposes in this module we will use `db = database(":memory:")` to create an in-memory database.

In [ ]:
#| export
def create_db(loc="static/infoflow.db"):
    db = database(loc)
    db.execute("PRAGMA foreign_keys = ON;")
    return db


::: {.callout-tip}
We can add foreign key constraints to the tables using the `transform` method from `sqlite_utils`.

```python
inf_tbl.transform(add_foreign_keys=[("<field_name>", "<table_name_to_connect>", "<field_name_in_table_to_connect>")])
```
:::

But for now we won't use foreign key constraints.

We can create the tables using the function `create_tables_from_pydantic` but we won't be using it. The tables will be created automatically when we insert the first item for each table.

In [ ]:
#| export
def create_tables_from_pydantic(db):
    sample_tool = Tool(name="Sample", organization_system=[OrganizationSystem.TAGS], phase_quality=PhaseQualityData(collect=PhaseQuality.GREAT, retrieve=PhaseQuality.BAD, consume=PhaseQuality.OK, extract=PhaseQuality.NA, refine=PhaseQuality.GREAT))
    sample_item = InformationItem(name="Sample", info_type=InformationType.WEB_ARTICLE, method=PhaseMethodData(collect=Method.MANUAL, retrieve=None, consume=None, extract=None, refine=None), toolflow=PhaseToolflowData(collect="Reader", retrieve="Recall", consume=None, extract=None, refine=None))
    sample_imp = Improvement(title="Sample", what="Test", why="Test", prio=1, tool="sample", phase=Phase.COLLECT)
    
    db["tools"].insert(sample_tool.flatten_for_db(), pk="slug")
    db["information_items"].insert(sample_item.flatten_for_db(), pk="slug") 
    db["improvements"].insert(sample_imp.flatten_for_db(), pk="slug")
    
    db["tools"].delete("sample")
    db["information_items"].delete("sample")
    db["improvements"].delete("sample")
    
    return db.t.tools, db.t.information_items, db.t.improvements

In [ ]:
tr = Tool(name="Sample", organization_system=[OrganizationSystem.TAGS], phase_quality=PhaseQualityData(collect=PhaseQuality.GREAT, retrieve=PhaseQuality.BAD, consume=PhaseQuality.OK, extract=PhaseQuality.NA, refine=PhaseQuality.GREAT))

In [ ]:
tr.flatten_for_db()

{'name': 'Sample',
 'collect': None,
 'retrieve': None,
 'consume': None,
 'extract': None,
 'refine': None,
 'slug': 'sample',
 'organization_system': '["tags"]',
 'collect_quality': 'great',
 'retrieve_quality': 'bad',
 'consume_quality': 'ok',
 'extract_quality': 'na',
 'refine_quality': 'great'}

Tests and usage examples

In [ ]:
db = create_db(":memory:")

In [ ]:
tool_tbl, inf_tbl, impr_tbl = create_tables_from_pydantic(db)

In [ ]:
tool_tbl()

[]

In [ ]:
inf_tbl.columns

[Column(cid=0, name='name', type='TEXT', notnull=0, default_value=None, is_pk=0),
 Column(cid=1, name='info_type', type='TEXT', notnull=0, default_value=None, is_pk=0),
 Column(cid=2, name='slug', type='TEXT', notnull=0, default_value=None, is_pk=1),
 Column(cid=3, name='collect_method', type='TEXT', notnull=0, default_value=None, is_pk=0),
 Column(cid=4, name='retrieve_method', type='TEXT', notnull=0, default_value=None, is_pk=0),
 Column(cid=5, name='consume_method', type='TEXT', notnull=0, default_value=None, is_pk=0),
 Column(cid=6, name='extract_method', type='TEXT', notnull=0, default_value=None, is_pk=0),
 Column(cid=7, name='refine_method', type='TEXT', notnull=0, default_value=None, is_pk=0),
 Column(cid=8, name='collect_toolflow', type='TEXT', notnull=0, default_value=None, is_pk=0),
 Column(cid=9, name='retrieve_toolflow', type='TEXT', notnull=0, default_value=None, is_pk=0),
 Column(cid=10, name='consume_toolflow', type='TEXT', notnull=0, default_value=None, is_pk=0),
 Colu

#### Add data to the database

Add the created instances to the SQLite tables

In [ ]:
Tool.get_instances()

{'testtool': Tool(name='TestTool', organization_system=[<OrganizationSystem.TAGS: 'tags'>], phase_quality=PhaseQualityData(collect=<PhaseQuality.GREAT: 'great'>, retrieve=<PhaseQuality.BAD: 'bad'>, consume=<PhaseQuality.OK: 'ok'>, extract=<PhaseQuality.NA: 'na'>, refine=<PhaseQuality.GREAT: 'great'>), collect=None, retrieve=None, consume=None, extract=None, refine=None, slug='testtool'),
 'sample': Tool(name='Sample', organization_system=[<OrganizationSystem.TAGS: 'tags'>], phase_quality=PhaseQualityData(collect=<PhaseQuality.GREAT: 'great'>, retrieve=<PhaseQuality.BAD: 'bad'>, consume=<PhaseQuality.OK: 'ok'>, extract=<PhaseQuality.NA: 'na'>, refine=<PhaseQuality.GREAT: 'great'>), collect=None, retrieve=None, consume=None, extract=None, refine=None, slug='sample')}

In [ ]:
InformationItem.get_instances()

{'test_article': InformationItem(name='Test Article', info_type=<InformationType.WEB_ARTICLE: 'web_article'>, method=PhaseMethodData(collect=<Method.MANUAL: 'manual'>, retrieve=None, consume=None, extract=None, refine=None), toolflow=PhaseToolflowData(collect=['Reader', 'Recall'], retrieve='Recall', consume=None, extract=None, refine=None), slug='test_article'),
 'sample': InformationItem(name='Sample', info_type=<InformationType.WEB_ARTICLE: 'web_article'>, method=PhaseMethodData(collect=<Method.MANUAL: 'manual'>, retrieve=None, consume=None, extract=None, refine=None), toolflow=PhaseToolflowData(collect='Reader', retrieve='Recall', consume=None, extract=None, refine=None), slug='sample')}

Add a single instance to the SQLite table

In [ ]:
impr_tbl.insert(Improvement.get_instances()['fix_search'].flatten_for_db())
inf_tbl.insert(InformationItem.get_instances()['test_article'].flatten_for_db())


{'name': 'Test Article',
 'info_type': 'web_article',
 'slug': 'test_article',
 'collect_method': 'manual',
 'retrieve_method': None,
 'consume_method': None,
 'extract_method': None,
 'refine_method': None,
 'collect_toolflow': '["Reader", "Recall"]',
 'retrieve_toolflow': 'Recall',
 'consume_toolflow': None,
 'extract_toolflow': None,
 'refine_toolflow': None}

In [ ]:
[r for r in inf_tbl.rows_where()]

[{'name': 'Test Article',
  'info_type': 'web_article',
  'slug': 'test_article',
  'collect_method': 'manual',
  'retrieve_method': None,
  'consume_method': None,
  'extract_method': None,
  'refine_method': None,
  'collect_toolflow': '["Reader", "Recall"]',
  'retrieve_toolflow': 'Recall',
  'consume_toolflow': None,
  'extract_toolflow': None,
  'refine_toolflow': None}]

Add multiple instances to the SQLite table

In [ ]:
tool_tbl.insert_all([t.flatten_for_db() for t in Tool.get_instances().values()])

<Table tools (name, collect, retrieve, consume, extract, refine, slug, organization_system, collect_quality, retrieve_quality, consume_quality, extract_quality, refine_quality)>

In [ ]:
[t.flatten_for_db() for t in Tool.get_instances().values()]

[{'name': 'TestTool',
  'collect': None,
  'retrieve': None,
  'consume': None,
  'extract': None,
  'refine': None,
  'slug': 'testtool',
  'organization_system': '["tags"]',
  'collect_quality': 'great',
  'retrieve_quality': 'bad',
  'consume_quality': 'ok',
  'extract_quality': 'na',
  'refine_quality': 'great'},
 {'name': 'Sample',
  'collect': None,
  'retrieve': None,
  'consume': None,
  'extract': None,
  'refine': None,
  'slug': 'sample',
  'organization_system': '["tags"]',
  'collect_quality': 'great',
  'retrieve_quality': 'bad',
  'consume_quality': 'ok',
  'extract_quality': 'na',
  'refine_quality': 'great'}]

In [ ]:
[r for r in tool_tbl.rows]

[{'name': 'TestTool',
  'collect': None,
  'retrieve': None,
  'consume': None,
  'extract': None,
  'refine': None,
  'slug': 'testtool',
  'organization_system': '["tags"]',
  'collect_quality': 'great',
  'retrieve_quality': 'bad',
  'consume_quality': 'ok',
  'extract_quality': 'na',
  'refine_quality': 'great'},
 {'name': 'Sample',
  'collect': None,
  'retrieve': None,
  'consume': None,
  'extract': None,
  'refine': None,
  'slug': 'sample',
  'organization_system': '["tags"]',
  'collect_quality': 'great',
  'retrieve_quality': 'bad',
  'consume_quality': 'ok',
  'extract_quality': 'na',
  'refine_quality': 'great'}]

#### Retrieve data from the database

Now retrieve the info from the database as intances from the Pydantic Dataclass

**Method 1:**

In [ ]:
db.t

improvements, information_items, tools

In [ ]:
type(db.t)

fastlite.core._TablesGetter

In [ ]:
db.t.tools()

[{'name': 'TestTool',
  'collect': None,
  'retrieve': None,
  'consume': None,
  'extract': None,
  'refine': None,
  'slug': 'testtool',
  'organization_system': '["tags"]',
  'collect_quality': 'great',
  'retrieve_quality': 'bad',
  'consume_quality': 'ok',
  'extract_quality': 'na',
  'refine_quality': 'great'},
 {'name': 'Sample',
  'collect': None,
  'retrieve': None,
  'consume': None,
  'extract': None,
  'refine': None,
  'slug': 'sample',
  'organization_system': '["tags"]',
  'collect_quality': 'great',
  'retrieve_quality': 'bad',
  'consume_quality': 'ok',
  'extract_quality': 'na',
  'refine_quality': 'great'}]

In [ ]:
tool = Tool.from_db(db.t.tools()[0])

In [ ]:
tool

Tool(name='TestTool', organization_system=[<OrganizationSystem.TAGS: 'tags'>], phase_quality=PhaseQualityData(collect=<PhaseQuality.GREAT: 'great'>, retrieve=<PhaseQuality.BAD: 'bad'>, consume=<PhaseQuality.OK: 'ok'>, extract=<PhaseQuality.NA: 'na'>, refine=<PhaseQuality.GREAT: 'great'>), collect=None, retrieve=None, consume=None, extract=None, refine=None, slug='testtool')

**Method 2:**

In [ ]:
tool_tbl()

[{'name': 'TestTool',
  'collect': None,
  'retrieve': None,
  'consume': None,
  'extract': None,
  'refine': None,
  'slug': 'testtool',
  'organization_system': '["tags"]',
  'collect_quality': 'great',
  'retrieve_quality': 'bad',
  'consume_quality': 'ok',
  'extract_quality': 'na',
  'refine_quality': 'great'},
 {'name': 'Sample',
  'collect': None,
  'retrieve': None,
  'consume': None,
  'extract': None,
  'refine': None,
  'slug': 'sample',
  'organization_system': '["tags"]',
  'collect_quality': 'great',
  'retrieve_quality': 'bad',
  'consume_quality': 'ok',
  'extract_quality': 'na',
  'refine_quality': 'great'}]

In [ ]:
tool_tbl()[0]

{'name': 'TestTool',
 'collect': None,
 'retrieve': None,
 'consume': None,
 'extract': None,
 'refine': None,
 'slug': 'testtool',
 'organization_system': '["tags"]',
 'collect_quality': 'great',
 'retrieve_quality': 'bad',
 'consume_quality': 'ok',
 'extract_quality': 'na',
 'refine_quality': 'great'}

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()